# 02 Machine Learning

## Proyecto: Credit Card Leads

### Equipo:
- Javier De La Rosa Mondragon
- Luis Fernando Merino Nambo
- Salomon Ramírez

![Leads](credit-cards.jpg)

###  Dataset

Credit Card Lead Prediction

https://www.kaggle.com/datasets/shelvigarg/credit-card-buyers

- ~250,000 datos en train set

- ~100,000 datos en test set (sin eqtiquetas)

- 10 Features
    - ID
    - Sexo
    - Edad
    - Región
    - Ocupación
    - Channel code (medio de comunicación para ofrecer la tarjeta)
    - Vintage (meses de historial crediticio)
    - Credit product (el cliente cuenta ya con un producto de crédito)
    - Balance promedio en cuenta
    - Es cleinte activo
    


- 1 Columna de target
    - Lead (aceptó la tarjeta de crédito ofecida)

## Objetivo

Analizar, limpiar y usar el train dataset para identificar clientes con tendencia a adquirur una tarjeta de crédito tomando como features las características del cliente.

Explorar las pistas que nos dan los diferentes modelos y tecnicas de clasificación para entender más cómo se pueden aplicar en diferentes casos y diferentes ámbitos.


## Retos

- El dataset que utlilzaremos es muy grande, lo cual requiere que encontremos soluciones para trabajar con él y el poder de cómputo requerido puede ser poco accesible para trabajarlo dentro de rangos de tiempo compatibles con otras actividades.
- De inmediato se puede observar que casi todas nuestras variables son categóricas. Para poder utlizarlas será necesario manipular los datos y buscar hiperparámetros adecuados para ellas (valores, distribuciones de probabilidad, etc.)

## Código

In [ ]:
# VARIABLES PARA EJECUCIÓN
DATA_FRACTION = .1 # parte del dataset completo de datos a utlizar
LONG_TIME_CELLS = True # ejecutar las celdas de EDA que tardan mucho
SMOTE_FRACTION = .05 # parte del dataset sobremuestreado a utlizar
DATA_TO_USE = 'over' # 'over' | 'sub' | 'orig'  # los datos que se usarán en el entrenamiento final

### Importación de bibliotecas 

In [ ]:
# numerical y Dfs
import pandas as pd
import numpy as np

# ML:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2 # selección de features categoricas
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

# SMOTE oversamplig
from imblearn.over_sampling import SMOTE

# árboles de decisión
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# visualización de árboles con dtreeviz
from sklearn import tree
#from dtreeviz import model
from dtreeviz.trees import *

# visualización de árboles con graphviz
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

# métricas de rendimiento
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# visualización
import seaborn as sns
from matplotlib import pyplot as plt

# timing operations
import time

In [ ]:
# paletas para seaborn
palQualitative = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palPaired = sns.color_palette("Accent")
sns.set()
sns.set_palette( palQualitative )

In [ ]:
def printTime(msg='time_now:',hours=1,mins=53):
    '''imprime la hora corregida porque el servidor la tiene mal'''
    current_time = time.localtime()
    new_time = current_time.tm_min - mins
    new_time = new_time % 60
    print(msg, f" {current_time.tm_hour-hours:02d}:{new_time:02d}:{current_time.tm_sec:02d}")

### Lectura de datos

In [ ]:
data = pd.read_csv('./data/train data credit card.csv')
print('Full Data size:',data.shape)

In [ ]:
# test data (como no tenemos la etiquetas, evaluaremos nuestros modelos con subsets para test)
TESTDATA = pd.read_csv('./data/test data credit card.csv')
TESTDATA.shape

In [ ]:
data.head()

### Limpieza de datos y EDA

In [ ]:
display(data.isna().sum())
print(data.shape)

In [ ]:
# limpieza de datos del set de entrenamiento
# ya que el dataset es muy grande, empezaremos eliminando las filas con datos vaciós
data = data.dropna()
data.reset_index(inplace=True,drop=True)
display(data.isna().sum().sum())
print(data.shape)

In [ ]:
def create_piechart_with_values(data, title, ax, optional_names=None):
    df = data
    value_counts = df.value_counts()
    category_counts = df.value_counts().values
    patches, texts, autotexts = ax.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%')

    total = sum(value_counts.values)
    label_values = []
    if optional_names:
        value_counts.index = optional_names
        
    for n,v in zip(value_counts.index,value_counts.values):
        v = f'{v:,}'
        #label_values.append(    f'{n}: {v} ({(v / total) * 100:.1f}%)'    )
        label_values.append(    f'{n}: {v}'    )
    
    for i, text in enumerate(texts):
        text.set_text(label_values[i])
    
    ax.set_title(title)

#### Variables Categóricas

In [ ]:
# ya que trabajaremos con clasificación, veamos cuántos casos hay de cada categoría:

fig, ax2 = plt.subplots(1, 1, figsize=(8, 5))
#sns.countplot(x=data.Is_Lead, ax=ax1)
#ax1.set_title('Countplot - Es Lead')
#for p in ax1.patches:
#    ax1.annotate(format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()), 
#                ha = 'center', va = 'center', xytext = (0, 5), textcoords = 'offset points')

create_piechart_with_values(data.Is_Lead, 'Piechart - Es Lead',ax2,['No Lead','Lead'])    

plt.tight_layout()
plt.show()


**Observaciones:**
    
Existe una desproporción muy grande entre las dos clases.

In [ ]:
# Veamos las distribuciones de algunas variables:
fig, axes = plt.subplots(3, 2, figsize=(12, 16))

#sns.countplot(data=creditCardLeadDf, x='Gender', ax=axes[0])
#axes[0].set_title('Sexo')
#sns.countplot(data=creditCardLeadDf, x='Is_Active', ax=axes[0])
#axes[0].set_title('Es cliente activo')
#sns.countplot(data=creditCardLeadDf, x='Credit_Product', ax=axes[1])
#axes[1].set_title('Cuenta con Producto de Crédito')
#sns.countplot(data=creditCardLeadDf, x='Occupation', ax=axes[1])
#axes[1].set_title('Ocupación')

create_piechart_with_values(data.Gender, 'Sexo',axes[0][0])
create_piechart_with_values(data.Occupation, 'Ocupación',axes[0][1])
create_piechart_with_values(data.Channel_Code, 'Channel Code',axes[1][0])
create_piechart_with_values(data.Credit_Product, 'Cuenta con Producto de Crédito',axes[1][1])
create_piechart_with_values(data.Is_Active, 'Es cliente activo',axes[2][0])
create_piechart_with_values(data.Is_Lead, 'Es Lead',axes[2][1])

#plt.tight_layout()
plt.show()

In [ ]:
# Veamos las proporciones en algunas variables:
print('Hay',data.Region_Code.value_counts().shape[0],'valores distintos de Region_Code')

fig, axes = plt.subplots(1, 1, figsize=(12, 4))
sns.countplot(data=data.sort_values(by='Region_Code'), x='Region_Code', ax=axes, order = data['Region_Code'].value_counts().index)
axes.set_xticklabels(axes.get_xticklabels(), rotation=90)
axes.set_title('Código de Región')
plt.tight_layout()

# Show the plot
plt.show()

**Observaciones:**

Más adelante podremos determinar si esta variable es relevante para la clasificación


#### Variables Numéricas

Veamos histogramas y boxplots para las variables numéricas que tienen nuestros datos

In [ ]:
def createHist(data,titulo,nobins=None):
    fig, axes = plt.subplots(2, 1, figsize=(12, 4))
    #data = creditCardLeadDf['Avg_Account_Balance']
    #titulo = 'Balance Promedio'

    if nobins:
        sns.histplot(data,ax=axes[0],bins= (data.max()-data.min()) )
    else:
        sns.histplot(data,ax=axes[0] )
    axes[0].set_xlabel('Values')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(titulo+' - Histograma')

    sns.boxplot(x=data, ax=axes[1])
    mean_value = np.mean(data)
    axes[1].plot(mean_value, 0,  marker='o', color='red', label='Mean')
    axes[1].text(mean_value+mean_value/1.5, -.3, f"Mean: {mean_value:.2f}", color='teal', ha='left', va='center')
    #axes[1].set_xlabel('Values')
    #axes[1].set_ylabel('Count')
    axes[1].set_title(titulo+' - Boxplot')
    #axes[1].legend()


    plt.tight_layout()
    plt.show()


In [ ]:
#sns.set_palette( palQualitative )
createHist(data['Avg_Account_Balance'],'Balance Promedio')
createHist(data['Age'],'Edad',nobins=True)
createHist(data['Vintage'],'Vintage',nobins=True)

In [ ]:
# ya que la columna ID no aporta nada a la clasificación, la eliminaremos
data = data.drop('ID', axis=1)

In [ ]:
data.head()

#### Conversión de variables a one-hot encoding

In [ ]:
# las siguientes columnas puedes ser codificadas como one-hot:
# Gender (2 valores), Region_Code (35), Occupation (4), Channel_Code (4), Credit_Product (2), Is_Active (2)

In [ ]:
# Codificamos en one-hot:
toOneHot = ['Gender', 'Occupation', 'Is_Active', 'Credit_Product','Channel_Code', 'Region_Code']
data = pd.get_dummies(data,columns=toOneHot)
print(data.shape)
display(data)

#### Eliminación de variables innecesarias

In [ ]:
# Para las variables que originalmente eran binarias, podemos eliminar una de sus
# dos columnas provenientes de get_dummies (Gender, Is_Active, ):
print(data.shape)
data = data.drop(['Gender_Female','Is_Active_No','Credit_Product_No'], axis=1)
print(data.shape)

#### Correlaciones entre variables y variables-salida

In [ ]:
# Tratemos de ver correlación entre las variables de Channel_Code y la salida
x_vars = [ x for x in list(data) if 'Channel_Code' in x] 
graphsPerRow = len(x_vars)
assert len(x_vars) % graphsPerRow == 0,f'usa un múltiplo de {len(x_vars)} para el número de gráficas por fila'
for i in range(int(len(x_vars)/graphsPerRow)):
    if LONG_TIME_CELLS:
        graph = sns.pairplot(data, x_vars=x_vars[i*graphsPerRow:i*graphsPerRow+graphsPerRow], y_vars='Is_Lead', height=5, aspect=1, kind='reg')
        graph.fig.suptitle('Correlación Channel-Salida', y=1.08)
    plt.show()

# Observaciones:    
# Parece que por lo menos las tres primeras sí tienen un efecto en la salida
# Aquí podría parecer que el chanel X4 es el menos efectivo obteniendo leads

In [ ]:
# Tratemos de ver correlación entre las variables de Occupation y la salida
x_vars = [ x for x in list(data) if 'Occupation' in x] 
graphsPerRow = len(x_vars)
assert len(x_vars) % graphsPerRow == 0,f'usa un múltiplo de {len(x_vars)} para el número de gráficas por fila'
for i in range(int(len(x_vars)/graphsPerRow)):
    if LONG_TIME_CELLS:
        graph = sns.pairplot(data, x_vars=x_vars[i*graphsPerRow:i*graphsPerRow+graphsPerRow], y_vars='Is_Lead', height=5, aspect=1, kind='reg')
        graph.fig.suptitle('Correlación Ocupación-Salida', y=1.08)
    plt.show()

# Observaciones:    
# Vemos que de entre las 4, Occupation_Entrepreneur es la que tiene más efecto sobre la salida

In [ ]:
# Tratemos de ver correlación entre nuestras variables numéricas y la salida:
x_vars = ['Age','Avg_Account_Balance','Vintage'] 
graphsPerRow = len(x_vars)
assert len(x_vars) % graphsPerRow == 0,f'usa un múltiplo de {len(x_vars)} para el número de gráficas por fila'
for i in range(int(len(x_vars)/graphsPerRow)):
    if LONG_TIME_CELLS:
        graph = sns.pairplot(data, x_vars=x_vars[i*graphsPerRow:i*graphsPerRow+graphsPerRow], y_vars='Is_Lead', height=5, aspect=1, kind='reg')
        graph.fig.suptitle('Correlación Variables Numéricas-Salida', y=1.08)
    plt.show()
    
# Observaciones:
# Vemos que de las tres tienen cierto grado apreciable de correlación con la salida

In [ ]:
# Y ahora veamos la correclación entre esas tres variables
#sns.pairplot(data, vars=x_vars, hue='Is_Lead', markers = ['d','v'], kind = "hist", diag_kind = None)
if LONG_TIME_CELLS:
    sns.pairplot(data, vars=x_vars, hue='Is_Lead', kind = "hist", diag_kind = 'kde')

# Observaciones:
# No encotramos mucha correlación entre éstas

In [ ]:
# También vemos la correlación entre estas variables usando Spearman
display(data[x_vars].corr(method="spearman"))

# Observaciones:
# Se observa una correlación significativa entre Age y Vintage, lo cual es de esperarse porque
# sólo la gente de mayor edad puede tener más historial crediticio
# (Yo esperaba encontrar más correlación entre edad y balance)

**Observaciones:**
    
Age y Vintage tienen correlación moderada.

In [ ]:
# Tratemos de ver correlaciones entre las variables de región y la salida
x_vars = list(data)
x_vars = [ x for x in x_vars if 'Region' in x] 

#print(x_vars)
#print(len(x_vars))

graphsPerRow = 5

assert len(x_vars) % graphsPerRow == 0,f'usa un múltiplo de {len(x_vars)} para el número de gráficas por fila'

if LONG_TIME_CELLS:
    for i in range(int(len(x_vars)/graphsPerRow)):
        sns.pairplot(data, x_vars=x_vars[i*graphsPerRow:i*graphsPerRow+graphsPerRow], y_vars='Is_Lead', height=5, aspect=1, kind='reg')
        plt.show()

# Observaciones:
# No notamos que alguna región destaque

In [ ]:
# Veamos si hay otras correlaciones entre nuestras variables:
thres = .4
corrs = data.corr(method="spearman")
print(f'Spearman > {thres}:')
print(f'-------------------------')
moreCorr = []
for i, row in corrs.iterrows():
    for ic,val in zip(list(data),row):
        if val>thres and i!=ic:
            if val>=.4 and val<.69:
                note = 'Correlación moderada'
            elif val>.7 and val<.89:
                note = 'Correlación fuerte'
            elif val>=.9:
                note = 'Correlación muy fuerte'
                
            variablesCorrelacionadas = [i,ic]
            variablesCorrelacionadas.sort()
            text = ' - '.join(variablesCorrelacionadas)
            moreCorr.append(f' {text} : {str(round(val,3))} ({note})')

moreCorr=list(set(moreCorr))
moreCorr.sort()
for i in moreCorr:
    print(i)

**Observaciones:**

Se nota una correlación moderada entre Channel_Code_X3 y  Vintage (pudo ser que ese canal se enfocó en ciertos clientes con mucha o poca antiguedad).

También moderada de ese mismo canal y la edad de los clientes.

Y correlaciones un poco más fuertes entre el canal X1 y clientes asalariados.

Desde luego, la correlación que ya habiamos notado entre edad y vintage (historial crediticio).

#### Preparación de datos para los modelos

In [ ]:
dataFull = data.copy()
X_full = dataFull.drop('Is_Lead',axis=1)
y_full = dataFull['Is_Lead']

In [ ]:
# Ejecutar desde aquí para diferentes pruebas con fracciones de los datos originales
data = dataFull.sample(frac=DATA_FRACTION)
data.reset_index(inplace=True,drop=True)

print("Fracción de datos utlizados para entrenamiento:", DATA_FRACTION*100,'%')
print('Train Data size:',data.shape)
print('Full Data size:',dataFull.shape)

#### Primer splitting para entrenar

In [ ]:
X = data.drop('Is_Lead',axis=1)
y = data['Is_Lead']

### Exploración de selección de features con SelectKBest

In [ ]:
# Selección de Features con Threshold fue descartada debido al alto número de variables categpóicas
best=10
selector = SelectKBest(k=best,score_func=chi2)

#scaler = MinMaxScaler()
#XScaled = scaler.fit_transform(X)
#XScaled = pd.DataFrame(data=XScaled, columns=list(X))

selector.fit(X,y)
print(f'Las {best} mejores features son:')
for col in selector.get_feature_names_out():
    print(f'   {col}')


**Observaciones:**

Las 3 variables numéricas son las más importantes de acuerdo al selector. También confirmamos que de las variables categóricas, Occupation_Entrepreneur juega el papel más importante importante.

### Clasificación con árboles de decisión

#### Arboles de decisión con datos originales (1/3)

In [ ]:
# dataframe para guardar performace metrics
treesPerformances = pd.DataFrame(columns=['Data', 'Acc','Recall','Precision', 'F1'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) # 70% training and 30% test

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [ ]:
# guardemos X_test y_test originales para usarlos en los dos árboles finales:
X_test_original = X_test.copy()
y_test_original = y_test.copy()

In [ ]:
importances_df = pd.DataFrame({'feature':list(X),'importancia':np.round(clf.feature_importances_,3)})
importances_df.sort_values(by='importancia',ascending=False,inplace=True)
display(importances_df.set_index('feature').head(10))

In [ ]:
pmAccuracy=round(accuracy_score(y_test_original,y_pred),3)
pmRecall=round(recall_score(y_test_original,y_pred),3)
pmPrecision=round(precision_score(y_test_original,y_pred),3)
pmF1=round(f1_score(y_test, y_pred),3)
treesPerformances.loc[len(treesPerformances.index)] = ['Datos originales', pmAccuracy, pmRecall, pmPrecision, pmF1 ] 
print(f"Accuracy: {pmAccuracy}")
print(f"Recall: {pmRecall}")
print(f"Precision: {pmPrecision}")

target_labels = ['No Lead','Lead']
plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test,y_pred)
ax = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',
            xticklabels=target_labels,
            yticklabels=target_labels)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
plt.show()

**Observaciones:**
    
Al parecer el hecho de que los ejemplos están desbalanceados affecta el desempeño del modelo. A continuación balancearemos los ejemplos por medio de sub/over sampling.


#### Árboles de decisión con datos submuestreados  (1/2)

In [ ]:
import imblearn

#Submuestreo

# lista los índices de Is_Lead
lead_idxs = data[data["Is_Lead"]==1].index.to_list()
print('Is_Lead == 1: ',len(lead_idxs))

# lista de índices que no son lead
nolead_idxs = data[data["Is_Lead"]==0].index.to_list()
print('Is_Lead == 0: ',len(nolead_idxs))

# seleccionamos aleatoriamente el doble de índices de no leads
random_nolead_idxs = np.random.choice(nolead_idxs, int(len(lead_idxs)*1.5), replace= False)
print('random picks len:',len(random_nolead_idxs))

# concatenamos los índices fraudulentos y normales y creamos el dataframe sub-sampleado
undersampled_indices = np.concatenate([lead_idxs, random_nolead_idxs])
undersampled_data = data.iloc[undersampled_indices,:]

print(f"Lead: {len(lead_idxs)}, Normales: {len(random_nolead_idxs)}")

print(f'Tamaño de data sin resampling: {data.shape}')
print(f'Tamaño de data luego de resampling: {undersampled_data.shape}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

create_piechart_with_values(data.Is_Lead, 'Datos Originales',ax1,['No','Sí'])    
create_piechart_with_values(undersampled_data.Is_Lead, 'Datos Submuestreados',ax2,['No','Sí'])    

plt.tight_layout()
plt.show()

In [ ]:
# "backup de data"
dataBkup = data.copy()

In [ ]:
X = undersampled_data.drop('Is_Lead',axis=1)
y = undersampled_data['Is_Lead']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) # 70% training and 30% test

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

importances_df = pd.DataFrame({'feature':list(X),'importancia':np.round(clf.feature_importances_,3)})
importances_df.sort_values(by='importancia',ascending=False,inplace=True)
display(importances_df.set_index('feature').head(10))

pmAccuracy=round(accuracy_score(y_test,y_pred),3)
pmRecall=round(recall_score(y_test,y_pred),3)
pmPrecision=round(precision_score(y_test,y_pred),3)
pmF1=round(f1_score(y_test, y_pred),3)
# no guardamos en df para metricas (haremos otro test con los datos originales, no los submuestreados)
print(f"Accuracy: {pmAccuracy}")
print(f"Recall: {pmRecall}")
print(f"Precision: {pmPrecision}")

target_labels = ['No Lead','Lead']
plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test,y_pred)
ax = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',
            xticklabels=target_labels,
            yticklabels=target_labels)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
plt.show()



In [ ]:
# test con los datos de prueba originales:
y_pred = clf.predict(X_test_original)

importances_df = pd.DataFrame({'feature':list(X),'importancia':np.round(clf.feature_importances_,3)})
importances_df.sort_values(by='importancia',ascending=False,inplace=True)
display(importances_df.set_index('feature').head(10))

pmAccuracy=round(accuracy_score(y_test_original,y_pred),3)
pmRecall=round(recall_score(y_test_original,y_pred),3)
pmPrecision=round(precision_score(y_test_original,y_pred),3)
pmF1=round(f1_score(y_test_original, y_pred),3)
treesPerformances.loc[len(treesPerformances.index)] = ['Sub-muestreo', pmAccuracy, pmRecall, pmPrecision, pmF1 ] 
print(f"Accuracy: {pmAccuracy}")
print(f"Recall: {pmRecall}")
print(f"Precision: {pmPrecision}")

target_labels = ['No Lead','Lead']
plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test_original,y_pred)
ax = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',
            xticklabels=target_labels,
            yticklabels=target_labels)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
plt.title('Matriz de confusión usando los datos originales (sin resampling)')    
plt.show()

#### Árboles de decisión con oversampling  (3/3)

In [ ]:
# restauramos data
data=dataBkup.copy()



print('Datos de entrada a SMOTE:',data.shape)

oversample = SMOTE()

X = data.drop('Is_Lead',axis=1)
y = data['Is_Lead']
X_oversampled, y_oversampled = oversample.fit_resample(X, y)


print("Datos SMOTE originales X, y:", X.shape,y.shape)

# lista los índices sampleados
smoteSampleIdxs = X.index.to_list()
print('smoteSampleIdx:',len(smoteSampleIdxs))
randomSmoteSampleIdxs = np.random.choice(smoteSampleIdxs, int(len(smoteSampleIdxs)*SMOTE_FRACTION), replace= False)
X_oversampled = X.iloc[randomSmoteSampleIdxs,:]
y_oversampled = y.iloc[randomSmoteSampleIdxs]

print("Fracción de datos SMOTE utlizados:", SMOTE_FRACTION*100,'%')
print(f"Matriz de features: {X_oversampled.shape}")
print(f"Matriz de etiquetas: {y_oversampled.shape}")

from collections import Counter
print(Counter(y_oversampled))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, test_size=0.3) # 70% training and 30% test

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

importances_df = pd.DataFrame({'feature':list(X),'importancia':np.round(clf.feature_importances_,3)})
importances_df.sort_values(by='importancia',ascending=False,inplace=True)
display(importances_df.set_index('feature').head(10))

pmAccuracy=round(accuracy_score(y_test,y_pred),3)
pmRecall=round(recall_score(y_test,y_pred),3)
pmPrecision=round(precision_score(y_test,y_pred),3)
pmF1=round(f1_score(y_test, y_pred),3)
# no guardamos en df de metricas, más adelante probaremos con los datos originales, no los sobremuestreados
print(f"Accuracy: {pmAccuracy}")
print(f"Recall: {pmRecall}")
print(f"Precision: {pmPrecision}")

target_labels = ['No Lead','Lead']
plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test,y_pred)
ax = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',
            xticklabels=target_labels,
            yticklabels=target_labels)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
plt.show()



In [ ]:
# falta visualización
viz = model(clf,
        tree_index=3,
        X_train=X_train,
        y_train=y_train,
        feature_names=list(X_train),
        target_name='Lead',
        class_names=['Lead','No Lead']
)

if LONG_TIME_CELLS:
    # no funciona
    # viz.view(scale=1.2,fontname='DejaVu Sans')
    pass


In [ ]:
text_representation = tree.export_text(decision_tree=clf)
print(X_train.columns[9])
print(text_representation)


In [ ]:
# falta visualización
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = list(X),class_names=['No Lead','Lead'])
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#graph.write_png('cardleads-tree.png')
#Image(graph.create_png())

In [ ]:
# test con los datos de prueba originales:
y_pred = clf.predict(X_test_original)

importances_df = pd.DataFrame({'feature':list(X),'importancia':np.round(clf.feature_importances_,3)})
importances_df.sort_values(by='importancia',ascending=False,inplace=True)
display(importances_df.set_index('feature').head(10))

pmAccuracy=round(accuracy_score(y_test_original,y_pred),3)
pmRecall=round(recall_score(y_test_original,y_pred),3)
pmPrecision=round(precision_score(y_test_original,y_pred),3)
pmF1=round(f1_score(y_test_original, y_pred),3)
treesPerformances.loc[len(treesPerformances.index)] = ['Sobre-muestreo', pmAccuracy, pmRecall, pmPrecision, pmF1 ] 
print(f"Accuracy: {pmAccuracy}")
print(f"Recall: {pmRecall}")
print(f"Precision: {pmPrecision}")

target_labels = ['No Lead','Lead']
plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test_original,y_pred)
ax = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',
            xticklabels=target_labels,
            yticklabels=target_labels)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
plt.title('Matriz de confusión usando los datos originales (sin resampling)')    
plt.show()

#### Comparación de métricas de rendimiento para los árboles de decisión usando modelos entrenados con diferentes sets de datos, probados todos con los datos originales

In [ ]:
display(treesPerformances.sort_values(by='Acc', ascending=False))

### Exploración de componentes pricipales

In [ ]:
# restauramos data
data=dataBkup.copy()
X = data.drop('Is_Lead',axis=1)
y = data['Is_Lead']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) # 70% training and 30% test

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=25)
X_red = pca.fit_transform(X_scaled)
#X_challenge_red = pca.transform(X_challenge_scaled)

print('Train data antes de reducción de dimensionalidad:',X.shape)
print('Train data luego de reducción de dimensionalidad:',X_red.shape)
#print('Test data luego de reducción de dimensionalidad:',X_challenge_red.shape)

xs = list(range(1, pca.explained_variance_ratio_.shape[0]+1))
capturedVarThreshold = .70

plt.figure(figsize=(12,5))
plt.bar(xs, pca.explained_variance_ratio_)
plt.xticks(xs)
plt.plot(xs, np.cumsum(pca.explained_variance_ratio_), '-*',color='red')
plt.grid()
plt.xlabel("Componentes principales")
plt.ylabel("Varianza explicada")

captured = np.cumsum(pca.explained_variance_ratio_)
#print(f"Varianza explicada por cada una de las componentes principales:\n{pca.explained_variance_ratio_}\n")
#print(f"Varianza acumulada explicada por cada una de las componentes principales:\n{captured)}\n")

plt.axhline(.5,color='gray',linestyle='-')
plt.axhline(.95,color='gray',linestyle='-')

maxPC = np.where(captured >= capturedVarThreshold)[0][0]
plt.axhline(capturedVarThreshold,color='gray',linestyle='--')
plt.axvline(maxPC,color='gray',linestyle='--')
#plt.xticks(rotation=80)

plt.title(f"{maxPC} componentes principales capturan {capturedVarThreshold*100}% de la varianza\n")

print(f"El {capturedVarThreshold*100}% de la varianza se obtiene con {maxPC} componentes principales\n")

### Preparación de pipeline y grid search para prueba de múltiples clasificadores con múltiples hiperparámetros

Usaremos los datos ~~submuestreados~~ sobremuestreados.

In [ ]:
if DATA_TO_USE == 'over':
    X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, test_size=0.3, random_state=101) # 70% training and 30% test
    X = X_oversampled
    y = y_oversampled
elif DATA_TO_USE == 'sub':
    X = undersampled_data.drop('Is_Lead',axis=1)
    y = undersampled_data['Is_Lead']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) # 70% training and 30% test
elif DATA_TO_USE == 'orig':
    data = dataBkup.copy()
    X = data.drop('Is_Lead',axis=1)
    y = data['Is_Lead']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101) # 70% training and 30% test

print(f"Usando data: {DATA_TO_USE}")    
print('X_train: {:,d}, X_test: {:,d}, y_train: {:,d}, y_test: {:,d}'.format(X_train.shape[0],X_test.shape[0],y_train.shape[0],y_test.shape[0]))

In [ ]:
# dataframe para guardar performace metrics
classifiersPerformances = pd.DataFrame(columns=['Classifier','Data',
                                                'Acc','Recall','Precision','F1','Roc-Auc',
                                               ])

In [ ]:
SCALER = True
POLY = False
DIMRED = True # si no hacemos dimRed el fitting toma demasiado tiempo
PCA_COMPONENTS = maxPC 

# CLASSIFIER /////////////////////(editar para cambiar el clasificador)/////////////////////
#classifier = RandomForestClassifier(n_jobs=-1)
classifier = SVC(probability=True)
#classifier =  RidgeClassifier()

# SCALER OBJECT
scaler = MinMaxScaler() if SCALER else None
#scaler = StandardScaler()

# POLYNOMIAL FEATURES OBJECT
poly = PolynomialFeatures(include_bias=True) if POLY else None

# DIMENSIONALITY REDUCTION OBJECT
dimRed = PCA(n_components=PCA_COMPONENTS) if DIMRED else None

# parámetros a probar con grid search:

# params for SVM:
#if type(classifier) == type(SVC()):
#    gridParams = [
#        {
#            'classifier__C': [.01,.1, 1, 10, 100, 1000],
#            'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#            'classifier__gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#            'poly__degree': [1,2,3]
#        },
#    ]
    

# Larger C values (e.g., 1, 10, 100) can be useful when:
#  The dataset is clean and has limited noise.
#  You want to capture complex decision boundaries.
#  You are willing to tolerate some misclassifications.

# Smaller gamma values (e.g., 0.001, 0.01) can be useful when:
#  The dataset is large.
#  The decision boundary is expected to be relatively smooth.
#  The classes are well-separated and the data is not too complex.

# params for SVM:
if type(classifier) == type(SVC()):
    gridParams = [
        {
            'classifier__C': [5, 10, 50],
            'classifier__kernel': ['poly', 'rbf', 'sigmoid'],
            'classifier__gamma': [.001,0.1, 0.5],
            'poly__degree': [1,3]
        },
    ]

# params for RandomForest:
if type(classifier) == type(RandomForestClassifier(n_jobs=-1)):
    gridParams = [
        {
            'classifier__n_estimators':[10,50,100]
        }]
    
# params for Linear(Ridge):
if type(classifier) == type(RidgeClassifier()):
    gridParams = [{
        'classifier__alpha':[.01,.1, 1,5,10],
    }]

# quitamos los parámetros que no se utilizarán
newParams = []
for d in gridParams:
    td = d.copy()
    for k in d.keys():
        if 'scaler' in k and not SCALER: del(td[k])
        if 'poly' in k and not POLY: del(td[k])
        if 'dimRed' in k and not DIMRED: del(td[k])
    newParams.append(td)
gridParams = newParams
            
# definimos los pasos del pipeline
steps=[
    ('scaler',scaler if scaler else 'passthrough'),
    ('dimRed', dimRed if dimRed else 'passthrough'),
    ('poly',poly if poly else 'passthrough'),
    ('classifier',classifier)
]

# quitamos del pipeline los pasos que no utilizaremos
steps = [s for s in steps if s[1]!='passthrough']
pipe = Pipeline(steps=steps)

# instanciamos un objeto de GridSearchCV
grid_search = GridSearchCV(
    estimator = pipe,
    param_grid = gridParams,
    scoring = 'accuracy',
    cv = 5, # incluimos cross validations en el grid search
    n_jobs = -1,
)

print('Pasos del pipeline:')
for s in [str(step) for step in pipe.steps]: print('\t',s)


### Búsqueda de hiperparámetros y entrenamiento del modelo

In [ ]:
# fit the model and time it
start_time = time.time()
#print('inicio:',time.asctime())
printTime('inicio:')
grid_search = grid_search.fit(X_train, y_train)
end_time = time.time()
duration = end_time - start_time
#print('fin:',time.asctime())
printTime('fin:')
print('Tiempo para fit operations:',round((duration)/60,2),'minutos')
print('Datos usados:',DATA_TO_USE)
print('X_train: {:,d}, X_test: {:,d}, y_train: {:,d}, y_test: {:,d}'.format(X_train.shape[0],X_test.shape[0],y_train.shape[0],y_test.shape[0]))

### Mejor estimador y métricas de rendimiento

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Fracción de datos utlizados para set inicial de entrenamiento:", DATA_FRACTION*100,'%')
print()
print('Datos usados para entrenamiento (originales, subsampled, oversampled):',DATA_TO_USE)
print('X_train: {:,d}, X_test: {:,d}, y_train: {:,d}, y_test: {:,d}'.format(X_train.shape[0],X_test.shape[0],y_train.shape[0],y_test.shape[0]))
print()
print('Tiempo para fit operations:',round((duration)/60,2),'minutos')
print()
print("Componentes principales utlizados",maxPC,f'({capturedVarThreshold*100}% de la varianza)')
print()
print("Mejor accuracy gridSearch:", round(best_accuracy,4))
print()
print("Hiperparámetros del mejor modelo de gridsearch:")
print(best_parameters)
print()
print('Pasos del pipeline:')
for s in [str(step) for step in pipe.steps]: print('\t',s)
print()

# creamos un figura con tres subplots (train, test, test original)
target_labels = ['No Lead','Lead']
fig, axes = plt.subplots(1, 3, figsize=(12, 3),sharey=True)


figR, axesR = plt.subplots(1, 3, figsize=(12, 5),sharey=True)


# train
y_pred = best_model.predict(X_train)
y_pred_proba = best_model.predict_proba(X_train)[:, 1]
pmScoreTrain=round(best_model.score(X_train,y_train),3)
pmAccuracyTrain=round(accuracy_score(y_train,y_pred),3)
pmRecallTrain=round(recall_score(y_train,y_pred),3)
pmPrecisionTrain=round(precision_score(y_train,y_pred),3)
pmF1Train=round(f1_score(y_train,y_pred),3)
pmRocAucTrain = roc_auc_score(y_train, y_pred_proba)

cm = confusion_matrix(y_train,y_pred)
ax=axes[0]
sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',xticklabels=target_labels,yticklabels=target_labels,ax=ax)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
ax.set_title('Train')

ax=axesR[0]
fpr, tpr, thresholds = roc_curve(y_train, y_pred_proba)
#plt.figure()
ax.set_title("Curva ROC (train)") 
ax.plot(fpr,tpr,color='red')
ax.plot([0,1],[0,1],linestyle='--',color='gray')
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")

    
    
    
    
    
# test
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
pmScoreTest=round(best_model.score(X_test,y_test),3)
pmAccuracyTest=round(accuracy_score(y_test,y_pred),3)
pmRecallTest=round(recall_score(y_test,y_pred),3)
pmPrecisionTest=round(precision_score(y_test,y_pred),3)
pmF1Test=round(f1_score(y_test,y_pred),3)
pmRocAucTest = roc_auc_score(y_test, y_pred_proba)

cm = confusion_matrix(y_test,y_pred)
ax=axes[1]
sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',xticklabels=target_labels,yticklabels=target_labels,ax=ax)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
ax.set_title('Test')

ax=axesR[1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
#plt.figure()
ax.set_title("Curva ROC (test)") 
ax.plot(fpr,tpr,color='red')
ax.plot([0,1],[0,1],linestyle='--',color='gray')
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")









# test con los datos de prueba originales:
#y_pred = best_model.predict(X_test_original)
y_pred = best_model.predict(X_full)
y_pred_proba = best_model.predict_proba(X_full)[:, 1]
#pmScoreTestOrig=round(best_model.score(X_test_original,y_pred),3)
pmScoreTestOrig=round(best_model.score(X_full,y_full),3)
#pmAccuracyTestOrig=round(accuracy_score(y_test_original,y_pred),3)
pmAccuracyTestOrig=round(accuracy_score(y_full,y_pred),3)
#pmRecallTestOrig=round(recall_score(y_test_original,y_pred),3)
pmRecallTestOrig=round(recall_score(y_full,y_pred),3)
#pmPrecisionTestOrig=round(precision_score(y_test_original,y_pred),3)
pmPrecisionTestOrig=round(precision_score(y_full,y_pred),3)
#pmF1TestOrig=round(f1_score(y_test_original, y_pred),3)
pmF1TestOrig=round(f1_score(y_full, y_pred),3)
pmRocAucTestOrig = roc_auc_score(y_full, y_pred_proba)

#cm = confusion_matrix(y_test_original,y_pred)
cm = confusion_matrix(y_full,y_pred)
ax=axes[2]
sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',xticklabels=target_labels,yticklabels=target_labels,ax=ax)
ax.set(xlabel='Predicted',ylabel='Real')
for t in ax.texts: t.set_text('{:,d}'.format(int(float(t.get_text()))))
ax.set_title('Test (dataset completo)')

ax=axesR[2]
fpr, tpr, thresholds = roc_curve(y_full, y_pred_proba)
#plt.figure()
ax.set_title("Curva ROC (dataset completo)") 
ax.plot(fpr,tpr,color='red')
ax.plot([0,1],[0,1],linestyle='--',color='gray')
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")







fig.suptitle('Matrices de confusión')
figR.suptitle('Curvas Roc-Auc')
plt.tight_layout()
plt.show()












classifiersPerformances.loc[len(classifiersPerformances.index)] = [type(pipe['classifier']),'Train',
                                                                  pmAccuracyTrain, pmRecallTrain, pmPrecisionTrain, pmF1Train,pmRocAucTrain
                                                                  ] 

classifiersPerformances.loc[len(classifiersPerformances.index)] = [type(pipe['classifier']),'Test',
                                                                  pmAccuracyTest, pmRecallTest, pmPrecisionTest, pmF1Test,pmRocAucTest
                                                                  ] 

classifiersPerformances.loc[len(classifiersPerformances.index)] = [type(pipe['classifier']),'Test Full',
                                                                  pmAccuracyTestOrig, pmRecallTestOrig, pmPrecisionTestOrig, pmF1TestOrig,pmRocAucTestOrig
                                                                  ] 

display(classifiersPerformances)

if type(classifier) == type(RandomForestClassifier(n_jobs=-1)):
    if SCALER:
        X_train_scaled = scaler.fit_transform(X_train)
        X_train = X_train_scaled
        
    if DIMRED:
        pca = PCA(n_components=maxPC)
        X_train_red = pca.fit_transform(X_train)
        X_train = X_train_red
    
    feature_cols = pd.DataFrame(X_train_red).columns.to_list()
    classifier.fit(X_train,y_train)
    
    importances = pd.DataFrame({'feature':feature_cols,'importancia':np.round(classifier.feature_importances_,3)})
    importances.sort_values(by='importancia',ascending=False,inplace=True)
    importances.set_index('feature')
    print(importances)



### Clustering con DBSCAN

In [ ]:
modelo = DBSCAN(eps=150, min_samples=12)
yhat = modelo.fit_predict(X)
clusters = [j for j in np.unique(yhat) if j!=-1]
print(clusters)

In [ ]:
XWithYAndCluster = pd.concat((X.reset_index(drop=True),y.reset_index(drop=True),pd.DataFrame(yhat,columns=['Cluster'])), axis=1)

print("Clientes que no fueron agregados a un cluster:",XWithYAndCluster[XWithYAndCluster.Cluster==-1].shape[0],'/',XWithYAndCluster.shape[0])

clientesEnClustersTotales=0
clientesEnClusters=[]
for cluster in clusters:
    clientesEnClusters.append( XWithYAndCluster[XWithYAndCluster.Cluster==cluster].shape[0] )
    clientesEnClustersTotales += XWithYAndCluster[XWithYAndCluster.Cluster==cluster].shape[0]

plt.figure(figsize=(12,5), dpi=300)
sns.barplot(x=np.arange(len(clientesEnClusters)), y=clientesEnClusters)
plt.xlabel("Cluster")
plt.ylabel("Clientes")
plt.title("Clientes en cada cluster")
plt.show()

print('El total de clientes en clusters es de:',clientesEnClustersTotales)
#display(XWithYAndCluster.head())
print('Estadísticas descriptivas del cluster 6:')
display(XWithYAndCluster[XWithYAndCluster.Cluster==6].describe())



**Observaciones:**

Los clientes dentro de cada cluster deben tener características similares. Revisando las desviaciones estándar de cada feature podemos hacernos cierta idea de las features que comparten:

#### Desviación estándar de cada feature de los datos del un par de clusters:

In [ ]:
clusterNumber = 5
stds=XWithYAndCluster[(XWithYAndCluster.Cluster==clusterNumber)]
stds.describe().loc['std'].sort_values()

In [ ]:
clusterNumber = 9
stds=XWithYAndCluster[(XWithYAndCluster.Cluster==clusterNumber)]
stds.describe().loc['std'].sort_values()

### Clustering con Jerárquico usando PCA

In [ ]:
# TODO: remove these X and y assigns (already in appropiate cell):
#X = X_oversampled
#y = y_oversampled

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=2)
X_red = pca.fit_transform(X_scaled)

explained_variance_ratios = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratios)
for i in range(2):
    print(f"Varianza explicada para el componente {i+1}: {explained_variance_ratios[i]}")
print(f"Varianza acumulada para {2} componentes: {cumulative_variance[2-1]}")

modelo = AgglomerativeClustering(distance_threshold=5.0,
                                 n_clusters=None,
                                 compute_full_tree=True)

#modelo = DBSCAN(eps=5000, min_samples=50) # no logré un clustering con DBSCAN sobre este dataset

yhat = modelo.fit_predict(X_red)
clusters = [j for j in np.unique(yhat) if j!=-1]
print("Clusters:")
print(clusters)


**Observaciones:**

No logré un clustering sobre estos datos usando DBSCAN

In [ ]:
XWithYAndCluster = pd.concat((X.reset_index(drop=True),y.reset_index(drop=True),pd.DataFrame(yhat,columns=['Cluster'])), axis=1)

print("Clientes que no fueron agregados a un cluster:",XWithYAndCluster[XWithYAndCluster.Cluster==-1].shape[0],'/',XWithYAndCluster.shape[0])

clientesEnClustersTotales=0
clientesEnClusters=[]
for cluster in clusters:
    clientesEnClusters.append( XWithYAndCluster[XWithYAndCluster.Cluster==cluster].shape[0] )
    clientesEnClustersTotales += XWithYAndCluster[XWithYAndCluster.Cluster==cluster].shape[0]

plt.figure(figsize=(12,5),dpi=300)
sns.barplot(x=np.arange(len(clientesEnClusters)), y=clientesEnClusters)
plt.xlabel("Cluster")
plt.ylabel("Clientes")
plt.title("Clientes en cada cluster")
plt.show()

print('El total de clientes en clusters es de:',clientesEnClustersTotales)
#display(XWithYAndCluster.head())
print('Estadísticas descriptivas del cluster 6:')
display(XWithYAndCluster[XWithYAndCluster.Cluster==6].describe())


#### Desviación estándar de cada feature de un cluster:

In [ ]:
clusterNumber = 7
stds=XWithYAndCluster[(XWithYAndCluster.Cluster==clusterNumber)]
stds.describe().loc['std'].sort_values()

**Observaciones:**

Podríamos encontrar qué cluster se basa más en cierta feature, Vintage, por ejemplo si comparamos la desviación estándar de cada cluster y encontramos cuál cluster tiene menos.

In [ ]:
# plotting de clusters
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4),dpi=300)
sns.scatterplot(x = X_red[:,1], y = X_red[:,0], hue = y, palette='deep', ax=axes[0], s=30)
axes[0].set_title('Is_Lead')
sns.scatterplot(x = X_red[:,1], y = X_red[:,0], hue = yhat==5, palette='deep', ax=axes[1], s=30)
axes[1].set_title('Cluster 5')
plt.suptitle(f'Clustering Jerárquico con 2CP ({round(cumulative_variance[-1],3)}% de la varianza)')
plt.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4),dpi=300)
sns.scatterplot(x = X_red[:,1], y = X_red[:,0], hue = yhat==7, palette='deep', ax=axes[0], s=30)
axes[0].set_title('Cluster 7')
sns.scatterplot(x = X_red[:,1], y = X_red[:,0], hue = yhat==11, palette='deep', ax=axes[1], s=30)
axes[1].set_title('Cluster 11')
plt.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4),dpi=300)
sns.scatterplot(x = X_red[:,1], y = X_red[:,0], hue = yhat==4, palette='deep', ax=axes[0], s=30)
axes[0].set_title('Cluster 4')
sns.scatterplot(x = X_red[:,1], y = X_red[:,0], hue = yhat==10, palette='deep', ax=axes[1], s=30)
axes[1].set_title('Cluster 10')
plt.tight_layout()
plt.show()

## Conclusiones

El trabajo realizado fue de inmensa utilidad para comprender más la implementación de muchos modelos y técnicas vistas en el módulo dos del diplomado.

El análisis de cluster puede revelar conocimientos de los datos que en primera instancia están muy ocultos.

El tamaño del dataset es un factor principal a tomar en cuenta al acercarse a un problema donde pueda aplicarse ML y en el caso del trabajo aquí presentado fue una de las carácteríticas más dificil de tratar para llegar a buenos modelos de clasificación.

Es necesario consultar continuamente la documentación de las implementaciones en Python así como la teoría detrás de cada modelo y algoritmos para familiarizarse plenamente con la materia.

El dataset y planteamiento del problema fue parte de un job-a-ton. La tabla de ranking de soluciones de varias pesonas se encuentra en https://datahack.analyticsvidhya.com/contest/job-a-thon-2/#LeaderBoard 

El máximo score alcanzado hasta ahora por la comunidad es de 0.8735155589